In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
seed = 42
tf.random.set_seed(seed)
rng = np.random.default_rng(seed=seed)

In [3]:
data = pd.read_csv('..\\Data\\ze41_mol_desc_db_red.csv', header=0, sep=';', decimal=',')

In [4]:
col_names = data.columns
x_cols = ['P_VSA_MR_5', 'Mor04m', 'E1p', 'CATS3D_02_AP', 'LUMO / eV']
X_full = data[col_names[3:]]
y = data[col_names[2]]

In [5]:
X_scaled = MinMaxScaler(feature_range=(0,1)).fit_transform(X_full)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.1, random_state=seed)

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation='relu', input_shape=(len(x_cols),)),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1)
])

In [8]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    loss='mean_squared_error')

In [9]:
weights = model.get_weights()

In [10]:
results = []

In [11]:
x_col_idxs = [col_names.tolist().index(x) for x in x_cols]
x_col_idxs

[370, 657, 791, 1236, 1258]

In [12]:
X_train_sel = X_train[:, x_col_idxs]
X_val_sel = X_valid[:, x_col_idxs]
history = model.fit(X_train_sel, y_train, validation_data=(X_val_sel, y_valid), verbose=0, epochs=100)
results.append({'idxs': x_col_idxs, 'loss': history.history['loss'][-1], 'val loss': history.history['val_loss'][-1]})

In [13]:
for _ in range(25):
    model.set_weights(weights)
    idxs = rng.choice(len(col_names)-3, size=len(x_cols), replace=False)
    X_train_sel = X_train[:, idxs]
    X_val_sel = X_valid[:, idxs]
    history = model.fit(X_train_sel, y_train, validation_data=(X_val_sel, y_valid), verbose=0, epochs=100)
    results.append({'idxs': idxs, 'loss': history.history['loss'][-1], 'val loss': history.history['val_loss'][-1]})

In [14]:
results

[{'idxs': [370, 657, 791, 1236, 1258],
  'loss': 0.03230303153395653,
  'val loss': 0.03562900424003601},
 {'idxs': array([972, 552, 823, 112, 545], dtype=int64),
  'loss': 0.045229025185108185,
  'val loss': 0.025419676676392555},
 {'idxs': array([1227,  118,  661,  959,  926], dtype=int64),
  'loss': 0.046594876796007156,
  'val loss': 0.03695283830165863},
 {'idxs': array([ 629, 1054,  566,  466,  230], dtype=int64),
  'loss': 0.04591944441199303,
  'val loss': 0.022357648238539696},
 {'idxs': array([ 685,  286,  567,  557, 1033], dtype=int64),
  'loss': 0.04047618433833122,
  'val loss': 0.016424372792243958},
 {'idxs': array([ 208, 1078, 1040,  348,  795], dtype=int64),
  'loss': 0.048129864037036896,
  'val loss': 0.03343585133552551},
 {'idxs': array([1123,  980,  560, 1219,  853], dtype=int64),
  'loss': 0.05517670884728432,
  'val loss': 0.02494923211634159},
 {'idxs': array([ 55, 625, 687, 936, 194], dtype=int64),
  'loss': 0.029710344970226288,
  'val loss': 0.04035470262169